# Treść zadania

In [1]:
# cel: zbudowac siec neuronową, ktora bedzie klasyfikowala artykul prasowy do jednej z 46 kategorii tematycznych
# dane: Reuters data
# ustalic liczbe slow, do ktorych ograniczymy reprezentacje
# wgrac zbior danych z tensorflow.keras.datasets (reuters, i jego metoda load_data())
# sprawdzic ksztalty tensorow
# obejrzec dane
# czyli sa to listy list (identyfikatorow slow, podobnie jak w imdb)
# dokonac krotkiej eksploracyjnej analizy danych
# czy sa same liczby 0-size, (czy nie ma np. znakow '?')
# obejrzec formę etykiet (ytrain, ytest)
# obejrzec jakie wartosci przyjmuje ytrain, ytest (czyli ile jest kategorii i jakie)
# obejrzec rozklad klas w ytrain i ytest (czy klasy sa zrownowazone?, czy ytrain i ytest maja podobne rozklady?)
# przetwarzanie danych
# zakodujmy wektory X jako "bag of words" (funkcja vectorize, tak jak w przykladzie z danymi imdb)
# napisac funkcję, ktora zakoduje tez etykiety jako 1-hot-encoding  ##
# (lub skorzystac w tym celu z gotowej funkcji to_categorical z tensorflow.keras.utils)
# zbudowac siec (interfejs "Sequential") o 2 warstwach ukrytych po 64 neurony (ile powinna miec neuronow 3. warstwa (wyjsciowa)?)
# jakich funkcji aktywacji nalezy uzyc w ukrytych a jakiej w warstwie wyjsciowej?
# skompilowac model (compile). Jakiej funkcji straty uzyc? (mozna jako metryki jakosci uzyc accuracy, dla upr.)
# wziac pierwszych 1000 obserwacji jako walidacyjne (podzielic odpowiednio zbiory na val i part)
# wytrenuj model na czesciowym zb. treningowym (20 epok, batch = 128) 
# przekazac dane walidacyjne do sledzenia historii uczenia, zeby dokonac early stopping
# uwaga: nie zapomniec, ze funkcja "fit" zwraca historię uczenia (zapamietac to w zmiennej)
# przeanalizuj historię uczenia (loss/accuracy na czesc.treningowym i walidacyjnym)
# wykonaj wykresy tych wielkosci jako funkcja epoki
# jaka liczba epok wyglada na optymalną?
# wytrenuj teraz model od poczatku na calym treningowym dla takiej liczby epok
# uwaga: nalezy od nowa zbudowac model, skompilowac go i wytrenowac 
#(samo wywolanie fit bedzie tylko kontynuowac uczenie na (przetrenowanym) starym modelu)
# dokonac ewaluacji na testowym
# dalsze eksperymentowanie:
# 1) eksperymentuj z kilkoma (min. 2) wariantami modelu (zmieniajac glebokosc, szerokosc warstw)
# np. co sie stanie jak zmniejszymy szerokosc 2. warstwy ukrytej? (tzw. bottleneck)
# 2) porownaj jak sobie poradzi jakis klasyczny klasyfikator na tych danych (ze scikit-learn)


##Notatki

##funkcja straty dla wielu klas sparse entropy???
##imie_nazwisko_sXXXXX_WUM_ZAD7.ipynb
##To categorical

# Cel
##### Zbudowac siec neuronową, ktora bedzie klasyfikowala artykul prasowy do jednej z 46 kategorii tematycznych.

# Dane
##### Wykorzystamy zbiór danych o nazwie Reuters. Zawiera on artykuły prasowe oraz ich przynależność do poszczególnych kategorii tematycznych. Zawiera on łącznie ponad 11 tysięcy artykułów prasowych. Każdy z nich przynależy do 1 z 46 kategorii tematycznych.

In [2]:
# impotwowanie potrzebnych bibliotek
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import plotly.express as px
from tensorflow import keras

# Trochę informacji o zbiorze danych reuters
 https://www.tensorflow.org/api_docs/python/tf/keras/datasets/reuters/load_data
##### Zbiór zawiera 11,228 wiadomości z Reuters. 
##### Słowom zostały nadane indeksy zgodne z częstotliwościa występowania w zbiorze. 5 najczęściej występujący wyraz oznaczony jest jako 5
##### num_words -  ile najczęściej występujących słów jest przetrzymywana

In [3]:
from tensorflow.keras.datasets import reuters

In [4]:
# Wczytywanie zbioru 
nwords = 10000
nseed = 10000
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = nwords, seed=nseed)

In [5]:
# Wymiary poszczególnych tensorów
print(list([o.shape, o.dtype] for o in [train_data, train_labels, test_data, test_labels]))

[[(8982,), dtype('O')], [(8982,), dtype('int64')], [(2246,), dtype('O')], [(2246,), dtype('int64')]]


# Wstępna ocena danych

In [6]:
# Typy tensorów
type(train_data), type(test_data)

(numpy.ndarray, numpy.ndarray)

In [7]:
# Podglądamy pojedynczy element listy
x = train_data[0]
type(x), len(x)

(list, 27)

In [8]:
# Unikalne wartości w przykładowym elemencie listy
np.unique(train_data[0])

array([   1,   12,   14,   17,   18,   26,   39,   44,   46,   53,   74,
         86,  147,  425, 4106, 4490, 5050, 6423])

In [9]:
# Unikale wartości kategorii
np.unique(train_labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], dtype=int64)

In [10]:
# Czy zbiór treningowy zawiera tyle samo etykiet i takie same etykiety co zbiór testowy 
np.unique(train_labels) == np.unique(test_labels)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [11]:
# Histogram dla da nych treningowych
fig = px.histogram(train_labels, labels={"value":"label"}, title="Histogram rozkładu klas dla danych treningowych")
fig.update_layout(showlegend=False)
fig.show()

In [12]:
# Histogram dla da nych testowych
fig = px.histogram(test_labels, labels={"value":"label"}, title="Histogram rozkładu klas dla danych testowych")
fig.update_layout(showlegend=False)
fig.show()

#### Oznaczenia etykiet znalazłem na stronie tensorflow (ściślej w dyskusji na githubie, do której link znalazłem na stronie tensorflow):
https://github.com/keras-team/keras/issues/12072
#### Oznaczenia klas:
https://raw.githubusercontent.com/SteffenBauer/KerasTools/master/Reuters_Analysis/reuters.dat


In [13]:
# Pobranie oznaczeń indeksów (jakie słowo onacza liczba)
index = reuters.get_word_index()

invertedIndex = dict([k, v] for (v,k) in index.items())

def decodeReview(wordlist):
    return " ".join([invertedIndex.get(i-3, '?') for i in wordlist])

In [14]:
# Przedstawienie 3 etykiet, wartości w liście oraz treść listy zamienioną na słowa 
# Trzy pierwsze (z największą częstotliwościa) wartości zostały zamienione na ?. 
# Najpopularniejsze slowa: the, of...
for i in range(3):
    print(f"Etykieta: {train_labels[i]}")
    print(f"Wartości w liście: {train_data[i]}")
    print(f"Zakodowanie liczb na słowa: {decodeReview(train_data[i])}")

Etykieta: 3
Wartości w liście: [1, 53, 46, 425, 26, 14, 74, 147, 26, 39, 46, 5050, 18, 14, 74, 4490, 18, 86, 44, 4106, 18, 14, 12, 6423, 18, 17, 12]
Zakodowanie liczb na słowa: ? shr loss 45 cts vs profit 25 cts net loss 641 000 vs profit 352 000 revs 5 747 000 vs 3 038 000 reuter 3
Etykieta: 4
Wartości w liście: [1, 3683, 81, 8, 16, 471, 284, 295, 277, 2046, 1367, 2910, 13, 1737, 482, 11, 15, 7, 197, 16, 8, 4, 1367, 50, 4190, 1737, 32, 315, 2, 7, 312, 643, 212, 62, 421, 22, 6589, 2, 9, 1581, 5, 8986, 1162, 4, 2, 2000, 6, 841, 1260, 2046, 1367, 471, 7, 4, 54, 109, 177, 4, 49, 8, 17, 12]
Zakodowanie liczb na słowa: ? southmark corp said it acquired 28 long term care facilities containing for approximately 70 mln dlrs in cash it said the facilities which contain approximately 2 500 ? in seven western states were bought from don ? and associates of salem ore the ? brings to 57 health care facilities acquired in the last three months the company said reuter 3
Etykieta: 19
Wartości w liście

# Wnioski z oceny danych
##### Dane są zawarte w liście list. Każda z list odpowiada artykułowi zapisanemu za pomocy liczb (słowa są zakodowane przy użyciu liczb, gdzie 1 odpowiada najczęściej występującemu słowu, 2 odpowiada drugiemu najczęściej występującemu słowu itd.).
##### Klasy nie są zrównoważone. Przeważa klasa 3 i 4. Według strony, którą zamieściłem powyżej klasa 3 oznacza: earn, natomiast 4 oznacza: acq. Klasy zostały podzielone pomiędzy testowe i treningowe z zachowaniem rozkładu klas.

# Przygotowywanie danych

In [15]:
# Zakodowanie danych jako bag of words
def vectorize(data, dimension = nwords):
    result = np.zeros((len(data), dimension))
    for i, l in enumerate(data):
        for w in l:
            result [i, w] = 1 
    return result

x_train = vectorize(train_data)
x_test = vectorize(test_data) 
x_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [16]:
# Wykorzystanie oneHotEncoding do zakodowania kategorii
# Z wartości 1-46 do listy 46 wartości z 1 wartością 1, reszta 0
from keras.utils import to_categorical

y_train = to_categorical(train_labels)
y_test =  to_categorical(test_labels)

print(y_train, y_test)
print(y_train.shape, y_test.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(8982, 46) (2246, 46)


# Modelowanie

In [17]:
# Model 3 warstw. Wszystkie elementy połączone ze sobą. 
# Funkcja dla warstw ukrytych Relu.
# Funkcja dla warstwy wyjściowej softmax. (wiele kategorii)
# Funkcja straty - categorical_crossentropy (wiele kategorii)
# Metryka - dokładność

from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation = "relu"),
    layers.Dense(64, activation = "relu"),
    layers.Dense(46, activation = "softmax")
])

model.compile(optimizer="rmsprop", metrics=["accuracy"], loss="categorical_crossentropy")

In [18]:
# Wydzielenie zbioru walidacyjnego ze zbioru treningowego 
x_val = x_train[:1000]
x_part_train = x_train[1000:]
y_val = y_train[:1000]
y_part_train = y_train[1000:]

In [19]:
# Ustawienie wartości epoch i batchsize
n_epochs = 20
n_batch = 128

In [20]:
# Trenowanie modelu na częściowym zbiorze dla zadanej ilości epok i batch
history = model.fit(x_part_train, y_part_train, epochs=n_epochs, batch_size=n_batch, validation_data=(x_val,y_val))


Epoch 1/20
63/63 [==============================] - 2s 17ms/step - loss: 1.8006 - accuracy: 0.6210 - val_loss: 1.2092 - val_accuracy: 0.7370
Epoch 2/20
63/63 [==============================] - 1s 9ms/step - loss: 0.9786 - accuracy: 0.7865 - val_loss: 1.0350 - val_accuracy: 0.7690
Epoch 3/20
63/63 [==============================] - 1s 10ms/step - loss: 0.6504 - accuracy: 0.8633 - val_loss: 0.8947 - val_accuracy: 0.8060
Epoch 4/20
63/63 [==============================] - 1s 9ms/step - loss: 0.4478 - accuracy: 0.9054 - val_loss: 0.8506 - val_accuracy: 0.8180
Epoch 5/20
63/63 [==============================] - 1s 10ms/step - loss: 0.3204 - accuracy: 0.9308 - val_loss: 0.9196 - val_accuracy: 0.8070
Epoch 6/20
63/63 [==============================] - 1s 12ms/step - loss: 0.2476 - accuracy: 0.9454 - val_loss: 0.9339 - val_accuracy: 0.8150
Epoch 7/20
63/63 [==============================] - 1s 9ms/step - loss: 0.2059 - accuracy: 0.9524 - val_loss: 0.9475 - val_accuracy: 0.8150
Epoch 8/20
63/63

In [21]:
# Przypisanie wartości history do zmiennej
history_dict = history.history

In [22]:
# Wypisanie kluczy w słowniku
for key, item in history_dict.items():
    print(key)

loss
accuracy
val_loss
val_accuracy


In [23]:
# Tworzenie wykresu
import plotly.graph_objects as go
def makeGraph(fname):
    # Przypisywanie wartości ze słownika
    train_values = history_dict[fname]
    val_values = history_dict["val_" + fname]
    epochs = list(range(1, len(val_values) + 1))
    # Tworzenie
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=epochs, y=train_values, mode='lines+markers', name=f'training {fname}'))
    fig.add_trace(go.Scatter(x=epochs, y=val_values, mode='lines+markers', name=f'validation {fname}'))
    fig.update_layout(title=f"Training vs validation {fname}")
    fig.show()


In [24]:
# Wykres dla funkcji straty
makeGraph("loss")

In [25]:
# Wykres dla dokładności
makeGraph("accuracy")

In [26]:
# Wyznaczamy epoke, po której zauważamy wzrost funkcji straty dla zbioru walidacyjnego
def which_epoch_loss(model_dict):
    for i in range(1, len(model_dict["val_loss"])):
        if(model_dict["val_loss"][i] > model_dict["val_loss"][i -1]):
            print(f'Do której epoki fukcja staty spada {i}')
            print(f'Wartość funkcji straty poprzedzająca wzrost: {model_dict["val_loss"][i-1]}, wartość funkcji straty, gdy zaczyna rosnąć: {model_dict["val_loss"][i] } ')
            break

# Wyznaczamy epoke, po której zauważamy spadek doskładności dla zbioru walidacyjnego
def which_epoch_accuracy(model_dict):
    for i in range(1, len(model_dict["val_accuracy"])):
        if(model_dict["val_accuracy"][i] < model_dict["val_accuracy"][i -1]):
            print(f'Do której epoki dokładność rośnie {i}')
            print(f'Wartość dokładności poprzedzająca spadek: {model_dict["val_accuracy"][i-1]}, wartość dokładności, gdy zaczyna spadać: {model_dict["val_accuracy"][i] } ')
            break

# Wyznacz minimalną wartość funkcji staty dla zbioru walidacyjnego
def get_min_loss_epoch(my_history):
    index_loss = my_history['val_loss'].index(min(my_history['val_loss']))
    index_loss += 1
    print(f'Minimalna wartość funkcji straty zbioru walidacyjnego dla epoki: {index_loss}')

# Wyznacz maksymalną wartość dokładności dla zbioru walidacyjnego
def get_max_accuracy_epoch(my_history):
    index_loss = my_history['val_accuracy'].index(max(my_history['val_accuracy']))
    index_loss += 1
    print(f'Maksymalna wartość dokładności zbioru walidacyjnego dla epoki: {index_loss}')

def execute_functions(my_history):
    which_epoch_loss(my_history)
    which_epoch_accuracy(my_history)
    get_min_loss_epoch(my_history)
    get_max_accuracy_epoch(my_history)

In [27]:
execute_functions(history_dict)

Do której epoki fukcja staty spada 4
Wartość funkcji straty poprzedzająca wzrost: 0.8505764603614807, wartość funkcji straty, gdy zaczyna rosnąć: 0.9195866584777832 
Do której epoki dokładność rośnie 4
Wartość dokładności poprzedzająca spadek: 0.8180000185966492, wartość dokładności, gdy zaczyna spadać: 0.8069999814033508 
Minimalna wartość funkcji straty zbioru walidacyjnego dla epoki: 4
Maksymalna wartość dokładności zbioru walidacyjnego dla epoki: 4


# Wyznaczenie optymalnej epoki
##### Zważywszy na powyższe wyniki oraz po przanalizowaniu wykresu stwierdzam, że optymalna epoka dla tego modelu to 4 epoka. Mimo małych zmian wartości w poszczególmych poźniejszych epokach dla dokładności, możemy zauważyć trend spadającej dokładności po epoce 4. Jeśli weźniemy pod uwagę fukcję straty to osiąga ona minimum w 4 epoce i ma tendencje do wzrostu w kolejnych epokach.
##### Dodatkowa uwaga - pomimo ustawienia seed optymalna epoka zmienia swoje wartość dla danego wywołania notatnika. Na podstawie paru wywołań wybrałem 4 jako najbardzie zliżoną wartość.

In [28]:
# Ustawienie epoki
n_epochs = 4

In [29]:
# Stworzenie modelu dla całego zbioru treningowego
model = keras.Sequential([
    layers.Dense(64, activation = "relu"),
    layers.Dense(64, activation = "relu"),
    layers.Dense(46, activation = "softmax")
])

model.compile(optimizer="rmsprop", metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(x_train, y_train, epochs=n_epochs, batch_size=n_batch)

Epoch 1/4
71/71 [==============================] - 1s 9ms/step - loss: 1.7356 - accuracy: 0.6260
Epoch 2/4
71/71 [==============================] - 1s 8ms/step - loss: 0.9043 - accuracy: 0.8043
Epoch 3/4
71/71 [==============================] - 1s 8ms/step - loss: 0.5983 - accuracy: 0.8753
Epoch 4/4
71/71 [==============================] - 1s 8ms/step - loss: 0.4124 - accuracy: 0.9130


# Ewaluacja na zbiorze testowym

In [30]:
# Ewaluacja
results = model.evaluate(x_test, y_test)
results

71/71 [==============================] - 2s 31ms/step - loss: 0.9208 - accuracy: 0.7934


[0.9208235144615173, 0.7934104800224304]

In [31]:
print(f'Wartość funkcji straty: {results[0]}')
print(f'Wartość dokładności: {results[1]}')

Wartość funkcji straty: 0.9208235144615173
Wartość dokładności: 0.7934104800224304


# Dalsze eksprerymenty

In [32]:
# Zwiększenie szerokości
model = keras.Sequential([
    layers.Dense(256, activation = "relu"),
    layers.Dense(256, activation = "relu"),
    layers.Dense(46, activation = "softmax")
])

model.compile(optimizer="rmsprop", metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(x_train, y_train, epochs=n_epochs, batch_size=n_batch)


results = model.evaluate(x_test, y_test)

print(f'Wartość funkcji straty: {results[0]}')
print(f'Wartość dokładności: {results[1]}')

Epoch 1/4
71/71 [==============================] - 2s 23ms/step - loss: 1.3865 - accuracy: 0.6980
Epoch 2/4
71/71 [==============================] - 2s 28ms/step - loss: 0.5832 - accuracy: 0.8730
Epoch 3/4
71/71 [==============================] - 2s 25ms/step - loss: 0.3269 - accuracy: 0.9274
Epoch 4/4
71/71 [==============================] - 0s 3ms/step - loss: 1.0538 - accuracy: 0.7796
Wartość funkcji straty: 1.0538424253463745
Wartość dokładności: 0.7796081900596619


In [33]:
# Dodanie dodatkowych warstwy

model = keras.Sequential([
    layers.Dense(64, activation = "relu"),
    layers.Dense(64, activation = "relu"),
    layers.Dense(64, activation = "relu"),
    layers.Dense(64, activation = "relu"),
    layers.Dense(46, activation = "softmax")
])

model.compile(optimizer="rmsprop", metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(x_train, y_train, epochs=n_epochs, batch_size=n_batch)


results = model.evaluate(x_test, y_test)

print(f'Wartość funkcji straty: {results[0]}')
print(f'Wartość dokładności: {results[1]}')

Epoch 1/4
71/71 [==============================] - 1s 9ms/step - loss: 1.7495 - accuracy: 0.6159
Epoch 2/4
71/71 [==============================] - 1s 10ms/step - loss: 1.0118 - accuracy: 0.7684
Epoch 3/4
71/71 [==============================] - 1s 9ms/step - loss: 0.6756 - accuracy: 0.8451
Epoch 4/4
71/71 [==============================] - 2s 26ms/step - loss: 1.1517 - accuracy: 0.7582
Wartość funkcji straty: 1.1516700983047485
Wartość dokładności: 0.7582368850708008


In [34]:
# Dodanie wartswy Dropout 


model = keras.Sequential([
    layers.Dense(64, activation = "relu"),
    layers.Dropout(0.25),
    layers.Dense(64, activation = "relu"),
    layers.Dense(46, activation = "softmax")
])

model.compile(optimizer="rmsprop", metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(x_train, y_train, epochs=n_epochs, batch_size=n_batch)


results = model.evaluate(x_test, y_test)

print(f'Wartość funkcji straty: {results[0]}')
print(f'Wartość dokładności: {results[1]}')

Epoch 1/4
71/71 [==============================] - 1s 7ms/step - loss: 1.9333 - accuracy: 0.5970
Epoch 2/4
71/71 [==============================] - 1s 8ms/step - loss: 1.1009 - accuracy: 0.7524
Epoch 3/4
71/71 [==============================] - 1s 8ms/step - loss: 0.8157 - accuracy: 0.8201
Epoch 4/4
71/71 [==============================] - 2s 30ms/step - loss: 0.9379 - accuracy: 0.7983
Wartość funkcji straty: 0.937860369682312
Wartość dokładności: 0.7983080744743347


In [35]:
# Dodanie wartswy Dropout i zmiana szerokości warstw


model = keras.Sequential([
    layers.Dense(256, activation = "relu"),
    layers.Dropout(0.25),
    layers.Dense(256, activation = "relu"),
    layers.Dense(46, activation = "softmax")
])

model.compile(optimizer="rmsprop", metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(x_train, y_train, epochs=n_epochs, batch_size=n_batch)


results = model.evaluate(x_test, y_test)

print(f'Wartość funkcji straty: {results[0]}')
print(f'Wartość dokładności: {results[1]}')

Epoch 1/4
71/71 [==============================] - 3s 28ms/step - loss: 1.4510 - accuracy: 0.6789
Epoch 2/4
71/71 [==============================] - 2s 26ms/step - loss: 0.6906 - accuracy: 0.8504
Epoch 3/4
71/71 [==============================] - 2s 25ms/step - loss: 0.4098 - accuracy: 0.9086
Epoch 4/4
71/71 [==============================] - 0s 3ms/step - loss: 1.0158 - accuracy: 0.7925
Wartość funkcji straty: 1.015790581703186
Wartość dokładności: 0.7925200462341309


In [39]:
# Bottleneck


model = keras.Sequential([
    layers.Dense(64, activation = "relu"),
    layers.Dense(16, activation = "relu"),
    layers.Dense(46, activation = "softmax")
])

model.compile(optimizer="rmsprop", metrics=["accuracy"], loss="categorical_crossentropy")
model.fit(x_train, y_train, epochs=n_epochs, batch_size=n_batch)


results = model.evaluate(x_test, y_test)

print(f'Wartość funkcji straty: {results[0]}')
print(f'Wartość dokładności: {results[1]}')

Epoch 1/4
71/71 [==============================] - 1s 9ms/step - loss: 2.2903 - accuracy: 0.5588
Epoch 2/4
71/71 [==============================] - 1s 7ms/step - loss: 1.2106 - accuracy: 0.7260
Epoch 3/4
71/71 [==============================] - 1s 8ms/step - loss: 0.9019 - accuracy: 0.7929
Epoch 4/4
71/71 [==============================] - 2s 24ms/step - loss: 1.0775 - accuracy: 0.7533
Wartość funkcji straty: 1.077487587928772
Wartość dokładności: 0.7533392906188965


In [37]:
# Klasyczny klasyfikator - bayes


from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

model_cc = MultinomialNB()

model_cc.fit(x_train, train_labels)
predicted_cc = model_cc.predict(x_test)

accuracy_score(test_labels, predicted_cc)

0.7524487978628673

In [38]:
# Klasyczny klasyfikator - drzewo


from sklearn.tree import DecisionTreeClassifier

model_cc = DecisionTreeClassifier()

model_cc.fit(x_train, train_labels)
predicted_cc = model_cc.predict(x_test)

accuracy_score(test_labels, predicted_cc)

0.6981300089047195

# Wnioski
##### Algorytmy klasycznego nauczania radzą sobie gorzej od sieci neuronowej w tym zadaniu. Drzewo dodatkowo jest bardzo wolne. 
##### Różne kombinacje sieci przeprowadzone w fazie eksperymentu zazwyczaj otrzymują gorsze wyniki lub porównywalne względem początkowej sieci (Dense 64, Dense 64, Dense 48). Dodanie warstwy Dropout zwraca obiecujące wyniki. Rezultatem dadania takiej warstwy jest porównywalny lub nieco lepszy wynik.